In [ ]:
!pip install duckdb -qq
!pip install ollama -qq

In [ ]:
import pandas as pd
import duckdb

In [ ]:
con = duckdb.connect('/content/drive/MyDrive/MFM project/bankruptcy_data.db')

In [ ]:
con.execute("SHOW TABLES").fetchdf()

,name
0,AAIIQ
1,AAMCF
2,ABILF
3,ABMC
4,ACNI
...,...
315,bankrupted
316,bankrupted_prob_results
317,financial2
318,financial3


In [ ]:
bankrupted_prob_results = con.execute("SELECT * FROM bankrupted_prob_results").fetchdf()
bankrupted = con.execute("SELECT * FROM bankrupted").fetchdf()

In [ ]:
bankrupted_prob_results.head(1)

,ticker,quarter_end,kmv_prob_default,zpp_prob_default
0,RNVA,2011-12-31,1.086395e-19,0.40908


In [ ]:
bankrupted.columns = ['date', 'ticker', 'company', 'available', 'enddate', 'startdate']

In [ ]:
bankrupted_prob_results = bankrupted_prob_results.merge(bankrupted[['ticker', 'company']], on='ticker')

In [ ]:
bankrupted_prob_results.head()

,ticker,quarter_end,kmv_prob_default,zpp_prob_default,company
0,RNVA,2011-12-31,1.086395e-19,0.40908,Rennova Health Inc
1,RNVA,2012-03-31,1.114426e-34,0.06974,Rennova Health Inc
2,RNVA,2012-03-31,0.000000e+00,0.06982,Rennova Health Inc
3,RNVA,2012-03-31,0.000000e+00,0.06976,Rennova Health Inc
4,RNVA,2012-06-30,0.000000e+00,0.00102,Rennova Health Inc


In [ ]:
uniques = bankrupted_prob_results[['ticker', 'company']].drop_duplicates()

In [ ]:
!sudo apt update
!sudo apt install pciutils lshw
!curl -fsSL https://ollama.com/install.sh | sh

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,851 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [4,976 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://ppa.launchp

In [ ]:
import subprocess
process = subprocess.Popen("ollama serve", shell=True)
!ollama pull gemma3:12b

In [ ]:
import time
import re
import json
from datetime import datetime
import ollama
from tqdm import tqdm

def get_prompt(ticker, company_name):
    return f"""
        You are a data collection specialist with expertise in comparing companies.
        Find from your trained knowledge for this [{ticker}] with company name [{company_name}]
        and extract 3 tickers with these criteria:
        1. did not bankrupted
        2. same industry with the given ticker
        3. same market capitalization with the given ticker.

        Return ONLY in this exact format:
        {ticker}|[ticker found 1]|[name of company for ticker found 1]
        {ticker}|[ticker found 2]|[name of company for ticker found 2]
        {ticker}|[ticker found 3]|[name of company for ticker found 3]
    """

def get_result(prompt, max_retries=3):
    for attempt in range(max_retries):
        try:
            time.sleep(2)
            response = ollama.chat(
                model='gemma3:12b',
                messages=[{'role': 'user', 'content': prompt}],
                think=False,
                options={
                    'temperature': 0,
                    'top_p': 0.1,
                    'repeat_penalty': 1.1
                }
            )
            return response['message']['content']
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt == max_retries - 1:
                return f"ERROR: Failed after {max_retries} attempts: {e}"
            time.sleep(5)

In [ ]:
uniques.tail()

,ticker,company
1146,VINC,Vincerx Pharma Inc
1165,SCPX,Scorpius Holdings Inc
1192,AIEV,Thunder Power Holdings Inc
1202,VRPX,Virpax Pharmaceuticals Inc
1219,MEHCQ,23andme Holding Co.


In [ ]:
# process = subprocess.Popen("ollama serve", shell=True)
# with open('/content/drive/MyDrive/MFM project/Data Preparation/non_bankrupted.txt', 'w') as outfile:
#     for ticker, company_name in tqdm(zip(uniques['ticker'], uniques['company'])):
#         prompt = get_prompt(ticker, company_name)
#         result = get_result(prompt)
#         print(result)
#         outfile.write(result + '\n')
#         outfile.flush()  # Force write to disk immediately

1it [00:36, 36.61s/it]

RNVA|HRC|[HCA Healthcare, Inc.]
RNVA|CHCT|[Community Health Systems, Inc.]
RNVA|MYL|[Mallinckrodt plc]


2it [00:51, 23.59s/it]

Okay, I will analyze Gex Management Inc. and find three companies meeting your criteria.

First, let's establish some baseline information about Gex Management Inc. (GXXM):

*   **Industry:** Investment Management (specifically, private equity and hedge fund management)
*   **Market Capitalization:** Approximately $1.38 Billion (as of late 2023/early 2024 - this can fluctuate)
*   **Bankruptcy Status:** Not Bankrupt

Now, I will search for companies matching these criteria.  Given the relatively specific requirements (industry, market cap, and no bankruptcy), finding exact matches is challenging. I'm prioritizing companies in the investment management/financial services space with a market cap close to $1.38 billion and a solid financial history.

Here are three potential matches:

GXXM|BLK|BlackRock, Inc.
GXXM|INV|Invesco Ltd.
GXXM|NTR|Natixis Investment Managers


3it [00:55, 14.86s/it]

...

Okay, I will analyze Thunder Power Holdings Inc. (AIEV) and find 3 companies meeting your criteria.

AIEV|SPWR|SunPower Corporation
AIEV|ENPH|Enphase Energy, Inc.
AIEV|SEDG|SolarEdge Technologies, Inc.


57it [05:45,  5.86s/it]

VRPX|ACOR|[Acorda Therapeutics, Inc.]
VRPX|CVIV|[Citius Pharmaceuticals, Inc.]
VRPX|PAST|[Paratek Pharmaceuticals, Inc.]


58it [05:50,  6.04s/it]

MEHCQ|CELH|Celsius Holdings, Inc.
MEHCQ|DNAE|Direxion Daily DNA ETF
MEHCQ|FWLB|Fulgent Genetics, Inc.


In [ ]:
with open('/content/drive/MyDrive/MFM project/Data Preparation/non_bankrupted.txt', 'r') as txtfile:
    lines = txtfile.readlines()

with open('result.txt', 'w') as outfile:
    for line in lines:
        if '|' not in line:
            pass
        else:
            cols = (line
                    .replace("[", "")
                    .replace("]", "")
                    .replace("\n", "")
                    .split("|")
                )

            outfile.write("|".join(cols[:3]) + "\n")

In [ ]:
non_bankrupted = pd.read_csv("/content/result.txt", delimiter="|", header=None)
non_bankrupted.columns = ['ticker', 'non_b_ticker', 'company_name']

In [ ]:
non_bankrupted.head()

,ticker,non_b_ticker,company_name
0,RNVA,HRC,"HCA Healthcare, Inc."
1,RNVA,CHCT,"Community Health Systems, Inc."
2,RNVA,MYL,Mallinckrodt plc
3,GXXM,BLK,"BlackRock, Inc."
4,GXXM,INV,Invesco Ltd.


In [ ]:
non_bankrupted = non_bankrupted.merge(bankrupted[['ticker', 'startdate', 'enddate']].drop_duplicates(), on = 'ticker', how = 'left')

In [ ]:
con.execute("DROP TABLE IF EXISTS non_bankrupted")
con.execute("CREATE TABLE non_bankrupted AS SELECT * FROM non_bankrupted")

In [ ]:
con.execute("SHOW TABLES").fetchdf()

,name
0,AAIIQ
1,AAMCF
2,ABILF
3,ABMC
4,ACNI
...,...
316,bankrupted_prob_results
317,financial2
318,financial3
319,financials


In [ ]:
non_bankrupted.head()

,ticker,non_b_ticker,company_name,startdate,enddate
0,RNVA,HRC,"HCA Healthcare, Inc.",2014-09-14,2018-09-14
1,RNVA,CHCT,"Community Health Systems, Inc.",2014-09-14,2018-09-14
2,RNVA,MYL,Mallinckrodt plc,2014-09-14,2018-09-14
3,GXXM,BLK,"BlackRock, Inc.",2015-06-10,2019-06-10
4,GXXM,INV,Invesco Ltd.,2015-06-10,2019-06-10


In [ ]:
!pip install duckdb

In [ ]:
import duckdb
import pandas
con = duckdb.connect('/content/drive/MyDrive/MFM project/bankruptcy_data.db')
df = con.execute("SELECT * FROM non_bankrupted").df()
df.head()

,ticker,non_b_ticker,company_name,startdate,enddate
0,RNVA,HRC,"HCA Healthcare, Inc.",2014-09-14,2018-09-14
1,RNVA,CHCT,"Community Health Systems, Inc.",2014-09-14,2018-09-14
2,RNVA,MYL,Mallinckrodt plc,2014-09-14,2018-09-14
3,GXXM,BLK,"BlackRock, Inc.",2015-06-10,2019-06-10
4,GXXM,INV,Invesco Ltd.,2015-06-10,2019-06-10


In [ ]:
df.tail(20)

,ticker,non_b_ticker,company_name,startdate,enddate
148,WGHTQ,REIC,Realty Income Corp.,2021-05-15,2025-05-15
149,WGHTQ,SPY,SPDR S&P 500 ETF,2021-05-15,2025-05-15
150,AXDXQ,LABU,Laboratory Corporation of America Holdings,2021-05-14,2025-05-14
151,AXDXQ,DGX,Diagnostic Imaging Services PLC,2021-05-14,2025-05-14
152,AXDXQ,IDXX,Idexx Laboratories Inc,2021-05-14,2025-05-14
153,VINC,ALXP,Allarity Therapeutics Inc.,2021-04-22,2025-04-22
154,VINC,MRTX,"Mirati Therapeutics, Inc.",2021-04-22,2025-04-22
155,VINC,IDXX,"Inotiv, Inc.",2021-04-22,2025-04-22
156,SCPX,AXA,AXA Group,2021-04-21,2025-04-21
157,SCPX,ALL,Allianz SE,2021-04-21,2025-04-21


In [ ]:
sorted = df.sort_values('non_b_ticker').reset_index(drop=True)

In [ ]:
sorted[sorted.non_b_ticker == 'IBM']

,ticker,non_b_ticker,company_name,startdate,enddate
86,IMCI,IBM,International Business Machines,2019-10-18,2023-10-18


In [ ]:
with open('/content/result (7).txt', 'r') as infile:
    lines = infile.readlines()

with open('output.txt', 'w') as outfile:
    for line in lines:
        if "|" not in line:
            pass
        else:
            cells = line.split("|")
            line = "|".join(cells[:7])
            outfile.write(line)

In [ ]:
import pandas as pd
data = pd.read_csv('output.txt', delimiter='|', header=None)

In [ ]:
data.iloc[:,0].nunique()

40

In [1]:
!pip install duckdb -qq

In [2]:
import pandas as pd
import duckdb
import yfinance as yf
import time

In [3]:
con = duckdb.connect('/content/drive/MyDrive/MFM project/bankruptcy_data.db')
non_bankrupted_financials = con.execute("SELECT * FROM financial4").fetchdf()
non_bankrupted_financials.head()

,ticker,date,shares,short_term_debt,long_term_debt,cash_equivalent,notes
0,CRSP,2018-06-30,47071836.0,18481000.0,69516000.0,322902016.0,Shares outstanding refers to shares outstandin...
1,CRSP,2018-03-31,46680816.0,14395000.0,69723000.0,344932000.0,Shares outstanding refers to shares outstandin...
2,CRSP,2017-12-31,40592248.0,14511.0,0.0,239758.0,"Shares outstanding are reported as 40,592,248...."
3,CPIX,2025-03-31,14961137.0,25519620.0,16383944.0,15108413.0,Shares represent the total number of shares ou...
4,CPIX,2024-12-31,13952624.0,31621728.0,10176170.0,17964184.0,Shares outstanding refers to the number of sha...


In [4]:
non_bankrupted_financials.ticker.unique()

array(['CRSP', 'CPIX', 'COST', 'COP', 'CNTY', 'CNET', 'CLX', 'CLSD',
       'CINF', 'CHK', 'CHCT', 'CELH', 'CEG', 'CDW', 'CDNS', 'CDMO', 'CCZ',
       'BIIB', 'BCSF', 'BC', 'BBBY', 'Acco', 'AXON', 'AVGO', 'ASRV',
       'APTO', 'APLD', 'ANF', 'AMGN', 'ALL', 'ALGN', 'ADXS', 'ACOR',
       'AAPL', 'ZS', 'XOM', 'WMT', 'WMB', 'WFC', 'VRSK', 'SUN', 'STX',
       'STEM', 'SQ', 'SPWR', 'SEDG', 'RUN', 'RGLD', 'REGN', 'QCOM',
       'POST', 'PLAY', 'PCAR', 'OKTA', 'NXTC', 'NXT', 'NXST', 'NXPI',
       'NTGR', 'NTAP', 'NEM', 'NBIX', 'MSFT', 'MRVL', 'MRVI', 'MRTX',
       'MRNA', 'MNST', 'LTC', 'LRCX', 'LITE', 'IRDM', 'IDXX', 'IBM',
       'HCP', 'GPS', 'GOOG', 'GMBL', 'GBR', 'FCX', 'FBNC', 'EXEL', 'EQIX',
       'ENPH', 'DXCM', 'DXC', 'DGX', 'DG', 'CVGW'], dtype=object)

In [5]:
for tick in non_bankrupted_financials.ticker.unique()[:1]:
    time.sleep(5)
    try:
        tmp = yf.download(tick, period='20y')[['Close']]
        tmp.reset_index(drop = False, inplace=True)
        tmp.columns = ['Date', 'Close']
        # con.execute(f"Drop Table If Exists {tick}")
        # con.execute(f"CREATE TABLE {tick} AS SELECT * FROM tmp")
    except:
        pass

/tmp/ipython-input-5-1253196934.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  tmp = yf.download(tick, period='20y')[['Close']]
[*********************100%***********************]  1 of 1 completed


In [8]:
tmp.Date.min(), tick

(Timestamp('2016-10-19 00:00:00'), 'CRSP')

In [ ]:
test = con.execute("Select * from GLAE").df()

In [ ]:
test.head()

,Date,Close
0,2015-07-27,7740.0
1,2015-07-28,7820.0
2,2015-07-29,7760.0
3,2015-07-30,7940.0
4,2015-07-31,8200.0


In [ ]:
test.Date.min()

Timestamp('2015-07-27 00:00:00')

In [ ]:
test.Date.max()

Timestamp('2025-07-25 00:00:00')

In [ ]:
con.close()